# CSV Context Manager Assignment

## Assignment 1

For this goal, you are given a number of CSV files (2), each of which have their first row with the field name.

You goal is to create a context manager that you can use to produce the data from each file in a named tuple with field names corresponding to the  header row field names.

You should use the `csv` module's `reader` function to help with parsing the data.

Your context manager should be generic in the sense that it should just need the file name, no other configuration or hardcoded functionality is required. You do not need to worry about data types for this goal - just return every field as a string.

In addition, your context manager should produce lazy iterators.

Implement this using a class that implements the context manager protocol

In [11]:
import csv
from collections import namedtuple

class CSVContextManager:
    def __init__(self, filename, delimiter=','):
        self.filename = filename
        self.delimiter = delimiter
        self.file = None
        self.reader = None
        self.RowType = None

    def __enter__(self):
        self.file = open(self.filename, 'r')
        self.reader = csv.reader(self.file, delimiter=self.delimiter)
        headers = next(self.reader)
        # Clean up header names to be valid identifiers
        clean_headers = [h.strip().replace(' ', '_').replace('-', '_') for h in headers]
        self.RowType = namedtuple('Row', clean_headers)
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        if self.file:
            self.file.close()
        return False

    def __iter__(self):
        return self

    def __next__(self):
        try:
            row = next(self.reader)
            return self.RowType(*row)
        except StopIteration:
            raise StopIteration()

### Example Usage

In [58]:
# Usage example

# Loading csv file with delimiter ;

with CSVContextManager('cars.csv', delimiter=';') as csv_context:
    i=0
    for row in csv_context:
        if i>=10: #limiting rows to 10 for clean display
            break 
        print(row)
        i+=1

Row(Car='Chevrolet Chevelle Malibu', MPG='18.0', Cylinders='8', Displacement='307.0', Horsepower='130.0', Weight='3504.', Acceleration='12.0', Model='70', Origin='US')
Row(Car='Buick Skylark 320', MPG='15.0', Cylinders='8', Displacement='350.0', Horsepower='165.0', Weight='3693.', Acceleration='11.5', Model='70', Origin='US')
Row(Car='Plymouth Satellite', MPG='18.0', Cylinders='8', Displacement='318.0', Horsepower='150.0', Weight='3436.', Acceleration='11.0', Model='70', Origin='US')
Row(Car='AMC Rebel SST', MPG='16.0', Cylinders='8', Displacement='304.0', Horsepower='150.0', Weight='3433.', Acceleration='12.0', Model='70', Origin='US')
Row(Car='Ford Torino', MPG='17.0', Cylinders='8', Displacement='302.0', Horsepower='140.0', Weight='3449.', Acceleration='10.5', Model='70', Origin='US')
Row(Car='Ford Galaxie 500', MPG='15.0', Cylinders='8', Displacement='429.0', Horsepower='198.0', Weight='4341.', Acceleration='10.0', Model='70', Origin='US')
Row(Car='Chevrolet Impala', MPG='14.0', Cy

In [50]:
with CSVContextManager('nyc_parking_tickets_extract.csv') as csv_context:
    i=0
    for row in csv_context:
        if i >= 10:  # Limit to the first 10 rows
         break
        print(row)
        i+=1


Row(Summons_Number='4006478550', Plate_ID='VAD7274', Registration_State='VA', Plate_Type='PAS', Issue_Date='10/5/2016', Violation_Code='5', Vehicle_Body_Type='4D', Vehicle_Make='BMW', Violation_Description='BUS LANE VIOLATION')
Row(Summons_Number='4006462396', Plate_ID='22834JK', Registration_State='NY', Plate_Type='COM', Issue_Date='9/30/2016', Violation_Code='5', Vehicle_Body_Type='VAN', Vehicle_Make='CHEVR', Violation_Description='BUS LANE VIOLATION')
Row(Summons_Number='4007117810', Plate_ID='21791MG', Registration_State='NY', Plate_Type='COM', Issue_Date='4/10/2017', Violation_Code='5', Vehicle_Body_Type='VAN', Vehicle_Make='DODGE', Violation_Description='BUS LANE VIOLATION')
Row(Summons_Number='4006265037', Plate_ID='FZX9232', Registration_State='NY', Plate_Type='PAS', Issue_Date='8/23/2016', Violation_Code='5', Vehicle_Body_Type='SUBN', Vehicle_Make='FORD', Violation_Description='BUS LANE VIOLATION')
Row(Summons_Number='4006535600', Plate_ID='N203399C', Registration_State='NY', 

In [51]:
with CSVContextManager('personal_info.csv') as csv_context:
    i=0
    for row in csv_context:
        if i >= 10:  # Limit to the first 10 rows
            break
        print(row)
        i+=1

Row(ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic')
Row(ssn='101-71-4702', first_name='Cayla', last_name='MacDonagh', gender='Female', language='Lao')
Row(ssn='101-84-0356', first_name='Nomi', last_name='Lipprose', gender='Female', language='Yiddish')
Row(ssn='104-22-0928', first_name='Justinian', last_name='Kunzelmann', gender='Male', language='Dhivehi')
Row(ssn='104-84-7144', first_name='Claudianus', last_name='Brixey', gender='Male', language='Afrikaans')
Row(ssn='105-27-5541', first_name='Federico', last_name='Aggett', gender='Male', language='Chinese')
Row(ssn='105-85-7486', first_name='Angelina', last_name='McAvey', gender='Female', language='Punjabi')
Row(ssn='105-91-5022', first_name='Moselle', last_name='Apfel', gender='Female', language='Latvian')
Row(ssn='105-91-7777', first_name='Audi', last_name='Roach', gender='Female', language='Estonian')
Row(ssn='106-35-1938', first_name='Mackenzie', last_name='Nussey', gender='Male'

## Assignment 2

The goal is to reproduce the work you did in Goal 1, but using a generator function and the `contextlib` `contextmanager` decorator.

In [26]:
import csv
from collections import namedtuple
from contextlib import contextmanager

@contextmanager
def csv_context_manager_decorator(filename, delimiter=','):
    """
    Context manager for CSV file yielding rows as namedtuples.
    """
    file = None
    try:
        file = open(filename, 'r')
        reader = csv.reader(file, delimiter=delimiter)
        headers = next(reader)
        # Clean up headers for valid identifiers
        clean_headers = [h.strip().replace(' ', '_').replace('-', '_') for h in headers]
        RowType = namedtuple('Row', clean_headers)
        # Yield rows as namedtuples
        yield (RowType(*row) for row in reader)
    finally:
        if file:
            file.close()

In [52]:
# Usage example:
with csv_context_manager_decorator('cars.csv', delimiter=';') as csv_data:
    i=0
    for row in csv_data:
        if i >= 10:  # Limit to the first 10 rows
            break
        print(row)
        i+=1

Row(Car='Chevrolet Chevelle Malibu', MPG='18.0', Cylinders='8', Displacement='307.0', Horsepower='130.0', Weight='3504.', Acceleration='12.0', Model='70', Origin='US')
Row(Car='Buick Skylark 320', MPG='15.0', Cylinders='8', Displacement='350.0', Horsepower='165.0', Weight='3693.', Acceleration='11.5', Model='70', Origin='US')
Row(Car='Plymouth Satellite', MPG='18.0', Cylinders='8', Displacement='318.0', Horsepower='150.0', Weight='3436.', Acceleration='11.0', Model='70', Origin='US')
Row(Car='AMC Rebel SST', MPG='16.0', Cylinders='8', Displacement='304.0', Horsepower='150.0', Weight='3433.', Acceleration='12.0', Model='70', Origin='US')
Row(Car='Ford Torino', MPG='17.0', Cylinders='8', Displacement='302.0', Horsepower='140.0', Weight='3449.', Acceleration='10.5', Model='70', Origin='US')
Row(Car='Ford Galaxie 500', MPG='15.0', Cylinders='8', Displacement='429.0', Horsepower='198.0', Weight='4341.', Acceleration='10.0', Model='70', Origin='US')
Row(Car='Chevrolet Impala', MPG='14.0', Cy

In [53]:
with csv_context_manager_decorator('personal_info.csv') as csv_data:
    i=0
    for row in csv_data:
        if i >= 10:  # Limit to the first 10 rows
            break
        print(row)
        i+=1

Row(ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic')
Row(ssn='101-71-4702', first_name='Cayla', last_name='MacDonagh', gender='Female', language='Lao')
Row(ssn='101-84-0356', first_name='Nomi', last_name='Lipprose', gender='Female', language='Yiddish')
Row(ssn='104-22-0928', first_name='Justinian', last_name='Kunzelmann', gender='Male', language='Dhivehi')
Row(ssn='104-84-7144', first_name='Claudianus', last_name='Brixey', gender='Male', language='Afrikaans')
Row(ssn='105-27-5541', first_name='Federico', last_name='Aggett', gender='Male', language='Chinese')
Row(ssn='105-85-7486', first_name='Angelina', last_name='McAvey', gender='Female', language='Punjabi')
Row(ssn='105-91-5022', first_name='Moselle', last_name='Apfel', gender='Female', language='Latvian')
Row(ssn='105-91-7777', first_name='Audi', last_name='Roach', gender='Female', language='Estonian')
Row(ssn='106-35-1938', first_name='Mackenzie', last_name='Nussey', gender='Male'

In [54]:
with csv_context_manager_decorator('nyc_parking_tickets_extract.csv') as csv_data:
    i=0
    for row in csv_data:
        if i>=10: # Limit to the first 10 rows
            break
        print(row)
        i+=1

Row(Summons_Number='4006478550', Plate_ID='VAD7274', Registration_State='VA', Plate_Type='PAS', Issue_Date='10/5/2016', Violation_Code='5', Vehicle_Body_Type='4D', Vehicle_Make='BMW', Violation_Description='BUS LANE VIOLATION')
Row(Summons_Number='4006462396', Plate_ID='22834JK', Registration_State='NY', Plate_Type='COM', Issue_Date='9/30/2016', Violation_Code='5', Vehicle_Body_Type='VAN', Vehicle_Make='CHEVR', Violation_Description='BUS LANE VIOLATION')
Row(Summons_Number='4007117810', Plate_ID='21791MG', Registration_State='NY', Plate_Type='COM', Issue_Date='4/10/2017', Violation_Code='5', Vehicle_Body_Type='VAN', Vehicle_Make='DODGE', Violation_Description='BUS LANE VIOLATION')
Row(Summons_Number='4006265037', Plate_ID='FZX9232', Registration_State='NY', Plate_Type='PAS', Issue_Date='8/23/2016', Violation_Code='5', Vehicle_Body_Type='SUBN', Vehicle_Make='FORD', Violation_Description='BUS LANE VIOLATION')
Row(Summons_Number='4006535600', Plate_ID='N203399C', Registration_State='NY', 

## Some tests to check functionality of context managers

In [55]:
import os
# Function to create a CSV file in a temporary location
def create_csv_file():
    csv_content = """Name, Age, City
John Doe,29,New York
Jane Smith,35,Los Angeles
Bob Johnson,40,Chicago"""
    
    csv_file = "test_sample.csv"
    with open(csv_file, "w") as f:
        f.write(csv_content)
    
    return csv_file

# Test class-based CSVContextManager in Jupyter
def test_csv_context_manager_class():
    csv_file = create_csv_file()
    
    expected_rows = [
        ('John Doe', '29', 'New York'),
        ('Jane Smith', '35', 'Los Angeles'),
        ('Bob Johnson', '40', 'Chicago')
    ]
    
    with CSVContextManager(csv_file) as rows:
        for i, row in enumerate(rows):
            assert row.Name == expected_rows[i][0]
            assert row.Age == expected_rows[i][1]
            assert row.City == expected_rows[i][2]
    
    print("Class-based CSVContextManager test passed")

# Test decorator-based csv_context_manager_decorator in Jupyter
def test_csv_context_manager_decorator():
    csv_file = create_csv_file()
    
    expected_rows = [
        ('John Doe', '29', 'New York'),
        ('Jane Smith', '35', 'Los Angeles'),
        ('Bob Johnson', '40', 'Chicago')
    ]
    
    with csv_context_manager_decorator(csv_file) as rows:
        for i, row in enumerate(rows):
            assert row.Name == expected_rows[i][0]
            assert row.Age == expected_rows[i][1]
            assert row.City == expected_rows[i][2]
    
    print("Decorator-based csv_context_manager_decorator test passed")


In [56]:
# Run both tests
test_csv_context_manager_class()
test_csv_context_manager_decorator()

Class-based CSVContextManager test passed
Decorator-based csv_context_manager_decorator test passed


In [57]:
# Cleanup the temporary CSV file
os.remove("test_sample.csv")

## Thank You